In [47]:
from typing import Dict, List
from langchain.chat_models import ChatOpenAI
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import PromptTemplate
from langchain.prompts.example_selector.base import BaseExampleSelector
        

chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ],
)

In [48]:
examples = [
    {
        "question": "What do you know about France?",
        "answer": """
        Here is what I know:
        Capital: Paris
        Language: French
        Food: Wine and Cheese
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Italy?",
        "answer": """
        I know this:
        Capital: Rome
        Language: Italian
        Food: Pizza and Pasta
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Greece?",
        "answer": """
        I know this:
        Capital: Athens
        Language: Greek
        Food: Souvlaki and Feta Cheese
        Currency: Euro
        """,
    },
]

In [53]:
from typing import Any, Dict

# 랜덤하게 examples에서 예제를 선택
class RandomExampleSelector(BaseExampleSelector):
    def __init__(self, examples):
        self.examples = examples
        
    def add_example(self, example):
        self.examples.append(example) # pass라고만 적어도 작동함
    
    def select_examples(self, input_variables):
        from random import choice
        return [choice(self.examples)]

In [54]:
# 예제들을 prompt를 사용해 형식화하기
example_prompt = PromptTemplate.from_template("Human:{question}\n AI:{answer}")

examples_selector = RandomExampleSelector(
    examples=examples,
)

# 형식화한 예제를 FewShowMPromptTemplate에 전달
# FewShowPromptTemplate는 각각의 예제들을 형식화하는 작업을 수행
prompt = FewShotPromptTemplate(
    example_prompt=example_prompt,
    example_selector=examples_selector,
    suffix="Human: What do you know about {country}?",
    input_variables=["country"],
)

prompt.format(country="Brazil")

'Human:What do you know about Greece?\n AI:\n        I know this:\n        Capital: Athens\n        Language: Greek\n        Food: Souvlaki and Feta Cheese\n        Currency: Euro\n        \n\nHuman: What do you know about Brazil?'